In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv("../data/datuk/files/datuk", sep='\t')

data

In [3]:
data

,from_content,types,to_content
0,ംടക്കൻ,{നാ.},"കിടുമ്മൻ, വാതിലിലും മറ്റും തറയ്ക്കുന്ന കുറ്റിയ..."
1,ംടക്കൻ,{നാ.},മുടക്കുന്നത്
2,അ1,{},"ഒരു ചുട്ടെഴുത്ത്, ദൂരെയുള്ള ഒന്നിനെ നിർദ്ദേശിക..."
3,അ1,{},"പേരെച്ചപ്രത്യയം. ഉദാ: ഒഴുകുന്ന, പറയുന്ന"
4,അ1,{},"അക്ഷര മാലയിലെ ആദ്യത്തെ അക്ഷരം, ഹ്രസ്വസ്വരം, കണ..."
...,...,...,...
148321,ഹർഷിത,{വി.},ഹർഷിക്കപ്പെട്ട
148322,ഹർഷുളം,{നാ.},മാൻ
148323,ഹർഷുളൻ,{നാ.},കാമുകൻ
148324,ഹർഷൻ,{നാ.},ഭാരതത്തിലെ ഒരു ചക്രവർത്തി


In [4]:
def remove_invisible(text):
    return re.sub(r'[\x00-\x1F\x7F\xAD]', '', text)


formatted_data = []
for idx, row in data.iterrows():
    word = ''.join(i for i in row["from_content"] if not i.isdigit())
    word = remove_invisible(word)
    for definition in row["to_content"].split(", "):
        formatted_data.append({"word": word, "definition":remove_invisible(definition)})

In [5]:
docs = [f"{entry['word']}: {entry['definition']}" for entry in formatted_data]

In [6]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v3")

ModuleNotFoundError: No module named 'custom_st'